In [1]:
#Campeonato Quantamental
#Pegando dados P/L
import requests
import pandas as pd
import io
import json
import re
import pandas as pd
from pandas_datareader import data, wb
import numpy as np


headers = {
    'authority': 'www.oceans14.com.br',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Mobile Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    # coloque aqui a url de qualquer canto que seja do site oceans14
    'referer': 'https://www.oceans14.com.br/acoes/itau-unibanco/itub/balanco-dividendos',
    'accept-language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7,la;q=0.6',
    'cookie': 'ASP.NET_SessionId=jypd15qvl1yu3ypbgqzecxjj; uid=0EAB27F50DB06E1775959795C4949E2D; origem=https://l.facebook.com/; contadorViewsAcoes=3',
}

params = (
    ######IDENTIFICAR TICKER DA ACAO######
    ('papel', 'ITUB3'),
    ('periodo', '10a'),
)

response = requests.get('https://www.oceans14.com.br/rendaVariavel/respostaAjax/gHistoricoPl.aspx', headers=headers, params=params)
dframe = pd.json_normalize(response.json())
jason = response.text
normal = json.loads(jason)
datas = []
valores = []
for row in normal["rows"]:
    date = row['c'][0]['v']
    formatedDate = re.search(r"(?<=Date\()[^\):]+(?=:?\))", date).group(0)
    simpleData = formatedDate.replace(',', '-').replace(' ', '')
    separatedData = simpleData.split('-')
    correctDataList = ''
    for number in separatedData:
        if number != '0':
            correctDataList += ("%02d" % int(number)) + "-"
    correctDataList = correctDataList[:-1]
    datas.append(correctDataList)
    valores.append(row['c'][1]['v'])
frameDict = {
    "Date": datas,
    "valores": valores
}
dframe = pd.DataFrame(data=frameDict)
df = pd.DataFrame()
df['valores'] = dframe['valores'].values.tolist()
valores = df['valores']

ImportError: No module named pandas_datareader

In [ ]:
#tickers = ['ITUB3.SA'] #IDENTIFICAR TICKER DA ACAO######
start_date = '2010-06-11'
end_date = '2020-06-08'
df = data.DataReader('ITUB3.SA', 'yahoo', start_date, end_date) #digitar ticker
df['Preco_Lucro'] = valores
tabela = df[['Close', 'Preco_Lucro']]
Mylist = tabela.columns.values.tolist()

In [ ]:
!pip install TA-Lib
import talib
import matplotlib.pyplot as plt
import sys

upperband, middleband, lowerband = talib.BBANDS(tabela['Preco_Lucro'], timeperiod=200, nbdevup=2, nbdevdn=2, matype=0)
tabela['upperband'] = upperband
tabela['middleband'] = middleband
tabela['lowerband'] = lowerband
tabela

In [2]:
#Logico

def buy_sell(df): 
  compras = []
  vendas = []
  #tickers = 'AAPL' ######IDENTIFICAR TICKER DA ACAO######
  flag = 3
  #flag = 1 comprado
  #flag = 0 vendeu

  for i in range(len(df)):
    if df[valores][i] < df[valores + '_binferior'][i]:   
      if flag != 1:
        compras.append(df[valores][i])
        vendas.append(np.nan)
        flag = 1
      else:
        compras.append(np.nan)
        vendas.append(np.nan)

    elif (df[valores][i] > df[valores + '_bsuperior'][i]):
      if ((flag != 0) & (flag != 3)):
        compras.append(np.nan)
        vendas.append(df[valores][i])
        flag = 0
      else:
        compras.append(np.nan)
        vendas.append(np.nan)

    else:
      compras.append(np.nan)
      vendas.append(np.nan)
        
  return (compras, vendas)

In [3]:
buy_sell = buy_sell(df)
df['compra'] = buy_sell[0]
df['venda'] = buy_sell[1]

NameError: name 'df' is not defined

In [4]:
#tickers = 'AAPL' ######IDENTIFICAR TICKER DA ACAO######


plt.figure(figsize=(15.7,4.7))
plt.plot(df[valores], label = 'Preço', alpha = 0.50)
plt.plot(df[valores + '_bsuperior'], label = 'Banda Supeferior',alpha = 0.5) 
plt.plot(df[valores + '_binferior'], label = 'Banda Inferior',alpha = 0.5)
plt.scatter(df.index, df['compra'], label = 'Buy', marker="1",color = 'green')
plt.scatter(df.index, df['venda'], label = 'Sell',marker="v",color = 'red')
plt.xlabel('2010-10-18 - 2019-12-31', fontsize =15)
plt.title("{}".format(tickers) + 'Preco Lucro', fontsize=15)
plt.legend(loc='upper left')


NameError: name 'plt' is not defined